# Install Required Software with Conda

**Create the Conda Software Environment for the opesarlab_OPERA-RTC-S1_Recipe_Book**

<div class="alert alert-success" style="display: flex; align-items: center; font-family: 'Times New Roman', Times, serif; background-color: 'rgba(200,0,0,0.2)'">
  <div style="width: 95%;">
    <h2><b>Important Notes About Your Jupyter Environment if not in OpenSARLab</b></h2>
    <b><i>Tip: Run the notebooks in this Jupyter Book from Jupyter Lab launched from a conda environment containing the required Jupyter packages. 
        <br/>
        This can be accomplished with the following commands:</i></b>
    <pre style="background-color: #f5f5f5; padding: 10px; border-radius: 5px; border: 1px solid #ccc; overflow: auto;">
      <code>mamba create -n jbook -c conda-forge jupyterlab notebook ipywidgets ipympl nb_conda_kernels</code>
      <code>mamba activate jbook</code>
      <code>python -m pip install jupyterlab-jupyterbook-navigation</code>
      <code>jupyter lab</code>
    </pre>
      Using this notebook, you will make a second environment containing the data access and science packages needed for the data recipes, but launch Jupyter Lab from an environment like the one provided above.
    <ul>
        <li>Jupyter selection widgets used in the notebooks require the packages <code>ipywidgets</code> and <code>notebook</code>.</li>
        <li>In order to use multiple conda environments in Jupyter Lab at the same time, you must install <code>nb_conda_kernels</code>.</li>
        <li>Interactive matplotlib plots requires the package <code>ipympl</code> in the environment running Jupyter Lab</li>
        <li>The code cell below will install packages if they are not found, but you will have to restart Jupyter Lab before continuing.</li>
    </ul>
  </div>
</div>


<div class="alert alert-info" style="display: flex; align-items: center; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div style="display: flex; align-items: center; width: 10%;">
    <a href="https://github.com/ASFOpenSARlab/opensarlab_OPERA-CSLC_Recipe_Book/issues">
      <img src="https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/logos/github_issues.png" alt="GitHub logo over the word Issues" style="width: 100%;">
    </a>
  </div>
  <div style="width: 95%;">
    <b>Did you find a bug? Do you have a feature request?</b>
    <br/>
    Explore GitHub Issues on this Jupyter Book's GitHub repository. Find solutions, add to the discussion, or start a new bug report or feature request: <a href="https://github.com/ASFOpenSARlab/opensarlab_OPERA-RTC-S1_Recipe_Book/issues">opensarlab_OPERA-RTC-S1_Recipe_Book Issues</a>
  </div>
</div>

<div class="alert alert-info" style="display: flex; align-items: center; justify-content: space-between; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div style="display: flex; align-items: center; width: 10%; margin-right: 10px;">
    <a href="mailto:uso@asf.alaska.edu">
      <img src="https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/logos/ASF_support_logo.png" alt="ASF logo" style="width: 100%">
    </a>
  </div>
  <div style="width: 95%;">
    <b>Have a question related to SAR, OPERA RTCs, or ASF data access?</b>
    <br/>
    Contact ASF User Support: <a href="mailto:uso@asf.alaska.edu">uso@asf.alaska.edu</a>
  </div>
</div>

---
## 0. Import Required Software and Set Some Paths

In [ ]:
from pathlib import Path
import json
import shutil
import subprocess
import yaml

try:
    import ipywidgets as widgets
    import nb_conda_kernels
    import notebook
except ModuleNotFoundError:
    !mamba install -c conda-forge notebook ipywidgets nb_conda_kernels --yes
    import ipywidgets as widgets
    
    print('\n\nIf the `notebook package` was installed, you may need to restart Jupyter Lab \n'
    'before widgets in the notebook will display correctly.')  

    print('\n\nIf the `nb_conda_kernels` package was installed, you may need to restart Jupyter Lab before the \n'
         '`osl_opera_cslc_jbook_env` environment created with this notebook will be available to use \n'
          'with the notebooks in this Jupyter Book that require it')

---
## 1. Indicate where you are running this Jupyter Book

In [ ]:
osl_option = widgets.RadioButtons(
    options=['In OpenSARLab', 'Not in OpenSARLab'],
    description='Where are you working?',
    disabled=False,
    layout=widgets.Layout(min_width='800px')
)
display(osl_option)

**Determine the conda prefix based on where you are working**

In [ ]:
env_config_dir = Path.cwd() / 'environments'

in_osl = 'In' in osl_option.value

if in_osl:
    conda_prefix = Path.home() / '.local/envs'
else:
    conda_info = subprocess.run(
        ['conda', 'run', '-n', 'base', 'conda', 'info', '--json'], 
        capture_output=True, 
        text=True
    )
    info = json.loads(conda_info.stdout)
    conda_prefix = Path(f"{info['root_prefix']}/envs")

---
## 2. Build the `osl_opera_rtc_s1_jbook_env` Conda Environment

**Select whether to create the environment from a config with explicitly pinned package versions or from an unpinned config**

After running the following code cell:
- Select `Pinned` to create an environment from a linux64 pinned yaml for maximum (but not guaranteed) stability when creating the environment on a 64 bit Linux system
- Select `Unpinned` to install the latest package versions on any system architecture

In [ ]:
config_options = widgets.RadioButtons(
    options=['Pinned', 'Unpinned'],
    description='Create the environment from pinned or unpinned configs',
    disabled=False,
    layout=widgets.Layout(min_width='800px')
)
display(config_options)

In [ ]:
pinned = 'Pinned' in config_options.value
if pinned:
    yaml_name = 'opensarlab_OPERA-RTC-S1_recipe_book_locked_env.yaml'
else:
    yaml_name = 'opensarlab_OPERA-RTC-S1_recipe_book_unlocked_env.yaml'

conda_yaml_pth = list(env_config_dir.glob(yaml_name))[0]
with open(conda_yaml_pth, 'r') as f:
    env = yaml.safe_load(f)
    env_name = env['name']

env_prefix = conda_prefix / env_name

print(f'environment name: {env_name}')
print(f'environment prefix: {env_prefix}')

**Clean your mamba and pip caches**

In [ ]:
!mamba clean -p -t --yes
!pip cache purge

**Create the environment, register its kernel, and delete the default kernel name**

In [ ]:
# Create the environment
!mamba env remove --prefix $env_prefix --yes
!mamba env create -f $conda_yaml_pth --prefix $env_prefix --yes

# Provide access to the environment in notebooks as a Jupyter kernel named for the environment 
!{env_prefix}/bin/python -m ipykernel install --prefix {env_prefix} --name {env_name}

# Having created a well-named kernel, delete the default kernel named 'Python'
shutil.rmtree(env_prefix/'share/jupyter/kernels/python3')